In [ ]:
!python -m pip install docx2pdf

In [6]:
import docx

def read_file(file):
    if isinstance(file, str):
        file_path = file
    else:
        file_path = file.name

    if file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        full_text = []
        for element in doc.element.body:
            if element.tag.endswith('p'):
                full_text.append(element.text)
            elif element.tag.endswith('tbl'):
                table = []
                for row in element.findall('.//w:tr', namespaces=element.nsmap):
                    cells = [cell.text for cell in row.findall('.//w:t', namespaces=element.nsmap)]
                    table.append(' | '.join(cells))
                full_text.append('\n'.join(table))
        return '\n'.join(full_text)
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

# Example usage
file_path = "/Users/huyknguyen/Desktop/redhorse/code_projects/ai_apps/docs/MSR/Eddy Biniam MSR Template.docx"
content = read_file(file_path)
print(content)

      



Monthly Status Report # --

HQ0034-20-F-0237
Task Order 4
Leased Facilities Division (LFD)
Administrative and Financial Services Support



For Work Performed:

September
2024 

Submitted to:

Mrs. Tina Hall  
Contracting Officer’s Representative
Washington Headquarters Service (WHS)
Acquisition Directorate (AD)
1155 Defense Pentagon Room 5B951
Washington, DC 
20301-1155
Tina.m.hall70.civ@mail.mil

(202) 819 - 2679

Submitted by:

Adrian Nicholas
Redhorse Corporation
1777 N. Kent St, Suite 1200
Arlington, VA 22209 
adrian.nicholas@redhorsecorp.com 
(347) 204-8125
Administrative and Financial Services Support Team:
Eddy Biniam 
Miguel Vega

Work Performed During MONTH YEAR 
Administrative Services Support

Financial Services Support
Assisted a colleague of mine in gathering necessary 3 DAI reports and 1 Maximo report in order to send out a Cost Transfer in a timely fashion.
Completed a hot item that my Supervisor and lead needed immediately. Did a contract PR’s for a CMTSS con

# openai audio output

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Is a golden retriever a good family dog?"
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("dog.wav", "wb") as f:
    f.write(wav_bytes)



In [ ]:
import base64
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to generate audio response
def generate_audio_response(messages):
    completion = client.chat.completions.create(
        model="gpt-4o-audio-preview",
        modalities=["text", "audio"],
        audio={"voice": "alloy", "format": "wav"},
        messages=messages
    )
    return completion.choices[0]

# Function to decode and save audio
def save_audio(audio_data, filename):
    wav_bytes = base64.b64decode(audio_data)
    with open(filename, "wb") as f:
        f.write(wav_bytes)

# Initialize conversation history
conversation_history = []

# Streamlit UI
st.title("Multi-turn Conversation Bot")

# User input
user_input = st.text_input("You: ", "")

if user_input:
    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Generate response
    response = generate_audio_response(conversation_history)

    # Add assistant message to conversation history
    conversation_history.append(response.message)

    # Save and play audio
    audio_filename = f"{response.message.audio.id}.wav"
    save_audio(response.message.audio.data, audio_filename)
    st.audio(audio_filename, format="audio/wav", autoplay=True)

    # Display transcript
    st.write("Assistant: ", response.message.audio.transcript)

# Display conversation history
st.subheader("Conversation History")
for message in conversation_history:
    if isinstance(message, dict):  # User message
        role = "You"
        content = message["content"]
    else:  # Assistant message
        role = "Assistant"
        content = message.audio.transcript
    st.write(f"{role}: {content}")

# elevenlab


In [ ]:
!python -m pip install elevenlabs

In [7]:

import os
import uuid
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)


def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="pNInz6obpgDQGcFmaJgB", # Adam pre-made voice
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2_5", # use the turbo model for low latency
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
        ),
    )

    # uncomment the line below to play the audio back
    # play(response)

    # Generating a unique file name for the output MP3 file
    save_file_path = f"{uuid.uuid4()}.mp3"

    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{save_file_path}: A new audio file was saved successfully!")

    # Return the path of the saved audio file
    return save_file_path

text_to_speech_file("Hello World")

ffe8e309-cb36-462a-a88d-fe7538a01d3a.mp3: A new audio file was saved successfully!


'ffe8e309-cb36-462a-a88d-fe7538a01d3a.mp3'